In [1]:
# ЯЧЕЙКА 1: Импорты
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm import tqdm

# Пути к директориям (при необходимости)
RMT_INFER_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/rmt_infer/"
BORZOI_BEDGRAPH_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/predicted_expression_by_chromosomes/"

print("Libraries imported. Paths set.")


Libraries imported. Paths set.


In [2]:
# Пути к parquet-файлам
labels_parquet = os.path.join(RMT_INFER_DIR, "labels.parquet")
labels_transcripts_parquet = os.path.join(RMT_INFER_DIR, "labels_transcripts.parquet")
preds_2seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_2seg.parquet")
preds_4seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_4seg.parquet")
preds_8seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_8seg.parquet")

# Загрузка
labels_df = pd.read_parquet(labels_parquet)
labels_transcripts_df = pd.read_parquet(labels_transcripts_parquet)
preds_2seg_df = pd.read_parquet(preds_2seg_parquet)
preds_4seg_df = pd.read_parquet(preds_4seg_parquet)
preds_8seg_df = pd.read_parquet(preds_8seg_parquet)

# Вывод краткой информации
print("labels_df:")
display(labels_df.head())
print("labels_df[preds].min/max/mean:\n", labels_df["preds"].min(), labels_df["preds"].max(), labels_df["preds"].mean())

print("\nlabels_transcripts_df:")
display(labels_transcripts_df.head())
print("labels_transcripts_df[preds].min/max/mean:\n",
      labels_transcripts_df["preds"].min(), 
      labels_transcripts_df["preds"].max(), 
      labels_transcripts_df["preds"].mean())

print("\npreds_2seg_df:")
display(preds_2seg_df.head())
print("preds_2seg_df[preds].min/max/mean:\n",
      preds_2seg_df["preds"].min(),
      preds_2seg_df["preds"].max(),
      preds_2seg_df["preds"].mean())

print("\npreds_4seg_df:")
display(preds_4seg_df.head())
print("preds_4seg_df[preds].min/max/mean:\n",
      preds_4seg_df["preds"].min(),
      preds_4seg_df["preds"].max(),
      preds_4seg_df["preds"].mean())

print("\npreds_8seg_df:")
display(preds_8seg_df.head())
print("preds_8seg_df[preds].min/max/mean:\n",
      preds_8seg_df["preds"].min(),
      preds_8seg_df["preds"].max(),
      preds_8seg_df["preds"].mean())


labels_df:


,chrom_pred,preds,chrom,start,end,start_pred,end_pred,Transcript stable ID,description_id
0,chr10,-4.60517,chr10,72893738,72893859,72893735,72893739,ENST00000334011,ENCFF123KIW
1,chr10,-4.60517,chr10,72893738,72893859,72893739,72893743,ENST00000334011,ENCFF123KIW
2,chr10,-4.60517,chr10,72893738,72893859,72893743,72893749,ENST00000334011,ENCFF123KIW
3,chr10,-4.60517,chr10,72893738,72893859,72893749,72893752,ENST00000334011,ENCFF123KIW
4,chr10,-4.60517,chr10,72893738,72893859,72893752,72893758,ENST00000334011,ENCFF123KIW


labels_df[preds].min/max/mean:
 -4.6051702 13.022035 1.0628820826820662

labels_transcripts_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,4.037295
1,ENST00000002596,ENCFF123KIWr,4.281860
2,ENST00000002829,ENCFF123KIW,5.388129
3,ENST00000009180,ENCFF123KIW,7.723541
4,ENST00000009589,ENCFF123KIWr,8.866660


labels_transcripts_df[preds].min/max/mean:
 -4.605170200000001 11.856355566803279 -0.4990853600718094

preds_2seg_df:


...

preds_2seg_df[preds].min/max/mean:
 -3.8326391907692305 6.641538864634146 -1.7986488347158898

preds_4seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,2.727709
1,ENST00000002596,ENCFF123KIWr,0.732307
2,ENST00000002829,ENCFF123KIW,2.426938
3,ENST00000009180,ENCFF123KIW,4.060136
4,ENST00000009589,ENCFF123KIWr,4.862711


preds_4seg_df[preds].min/max/mean:
 -4.2842083040022745 7.001022550393701 -0.715264122164497

preds_8seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,3.736615
1,ENST00000002596,ENCFF123KIWr,1.674160
2,ENST00000002829,ENCFF123KIW,0.736134
3,ENST00000009180,ENCFF123KIW,4.146430
4,ENST00000009589,ENCFF123KIWr,5.075033


preds_8seg_df[preds].min/max/mean:
 -4.180466452112676 6.389540544094489 -0.6615956298938233


In [3]:
# Убираем столбец preds, который нам не нужен (по вашим требованиям)
# Оставляем только chrom, start, end, Transcript stable ID, description_id
labels_clear_df = labels_df[['chrom', 'start', 'end', 'Transcript stable ID', 'description_id']].copy()

# Удаляем дубликаты по Transcript stable ID
labels_clear_df.drop_duplicates(subset=['Transcript stable ID'], inplace=True)

print("labels_clear_df (очищенная):")
display(labels_clear_df.head())
print("Shape:", labels_clear_df.shape)


labels_clear_df (очищенная):


,chrom,start,end,Transcript stable ID,description_id
0,chr10,72893738,72893859,ENST00000334011,ENCFF123KIW
379,chr10,73071294,73071357,ENST00000431293,ENCFF123KIW
425,chr10,73110454,73110567,ENST00000357321,ENCFF123KIW
772,chr10,73168118,73168386,ENST00000242505,ENCFF123KIW
1660,chr10,73247359,73247383,ENST00000440197,ENCFF123KIW


Shape: (4419, 5)


In [4]:
# ЯЧЕЙКА 4: Создание словаря сопоставлений description_id с файлом для bedGraph

# Создадим таблицу для соответствия description_id и имен файлов
experiment_mapping = {
    "ENCFF123KIW": "borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph",
    "ENCFF123KIWr": "borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph",
    "ENCFF784MDF": "borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph",
    "ENCFF784MDFr": "borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph",
    "ENCFF236XOK": "borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph",
    "ENCFF236XOKr": "borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph",
    "ENCFF781TTC": "borzoi_rnaseq_fold4_ENCSR071DYD+_fw.bedGraph",
    "ENCFF781TTCr": "borzoi_rnaseq_fold4_ENCSR071DYD-_fw.bedGraph",
    "ENCFF242BWW": "borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph",
    "ENCFF242BWWr": "borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph"
}

# Выводим для проверки
print("Словарь сопоставлений для description_id:")
for key, value in experiment_mapping.items():
    print(f"{key}: {value}")

Словарь сопоставлений для description_id:
ENCFF123KIW: borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph
ENCFF123KIWr: borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph
ENCFF784MDF: borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph
ENCFF784MDFr: borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph
ENCFF236XOK: borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph
ENCFF236XOKr: borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph
ENCFF781TTC: borzoi_rnaseq_fold4_ENCSR071DYD+_fw.bedGraph
ENCFF781TTCr: borzoi_rnaseq_fold4_ENCSR071DYD-_fw.bedGraph
ENCFF242BWW: borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph
ENCFF242BWWr: borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph


In [5]:
# ЯЧЕЙКА 5: Загрузка bedGraph файлов в оперативную память (fw вариант)

bedgraph_files = list(set(experiment_mapping.values()))
bedgraph_data = {}

for fname in tqdm(bedgraph_files, desc="Обработка файлов"):
    full_path = os.path.join(BORZOI_BEDGRAPH_DIR, fname)
    if not os.path.exists(full_path):
        print("Файл не найден:", fname)
        continue
    df_tmp = pd.read_csv(full_path, sep="\t", header=None, names=["chrom","start","end","value"])
    bedgraph_data[fname] = df_tmp
    
    # Выводим head/min/max/mean
    print(f"\nЗагружен файл {fname}, shape={df_tmp.shape}")
    display(df_tmp.head(3))
    print("value: min=%.3f max=%.3f mean=%.3f" % (df_tmp["value"].min(), df_tmp["value"].max(), df_tmp["value"].mean()))


Обработка файлов:   0%|                                                                                                                                          | 0/10 [00:00<?, ?it/s]


Загружен файл borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph, shape=(10799537, 4)


,chrom,start,end,value
0,chr10,72890684,72890716,0.538787
1,chr10,72890716,72890748,0.216714
2,chr10,72890748,72890780,0.051275


Обработка файлов:  10%|█████████████                                                                                                                     | 1/10 [00:03<00:32,  3.59s/it]

value: min=0.000 max=4004.835 mean=1.180

Загружен файл borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph, shape=(10799537, 4)


...

Обработка файлов:  90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████             | 9/10 [00:58<00:06,  6.40s/it]

value: min=0.000 max=4382.543 mean=1.627

Загружен файл borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph, shape=(10799537, 4)


,chrom,start,end,value
0,chr10,72890684,72890716,0.141283
1,chr10,72890716,72890748,0.042791
2,chr10,72890748,72890780,0.006480


Обработка файлов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:05<00:00,  6.54s/it]

value: min=0.000 max=9742.334 mean=2.038


In [6]:
PSEUDOCOUNT = 0.01
#base = 2 #основание

bedgraph_data_log = {}

for fname, df in bedgraph_data.items():
    df_log = df.copy()
    df_log["value"] = np.log(df_log["value"] + PSEUDOCOUNT)
    #df_log["value"] = np.log(df_log["value"] + PSEUDOCOUNT) / np.log(base)
    
    bedgraph_data_log[fname] = df_log
    
    # Выводим статистику
    print(f"\nЛогарифмирован файл {fname}")
    print("value(min/max/mean) =", 
          df_log["value"].min(), 
          df_log["value"].max(), 
          df_log["value"].mean())



Логарифмирован файл borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph
value(min/max/mean) = -4.604050246033248 8.29526008641543 -2.782012479944866

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph
value(min/max/mean) = -4.603217062228037 8.263734575941122 -2.3667830958282927

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph
value(min/max/mean) = -4.603812122672792 8.33599800939897 -2.4426745504733556

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph
value(min/max/mean) = -4.603574056001573 8.070751273337727 -2.7398935592542215

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph
value(min/max/mean) = -4.604979469312392 8.779723568889084 -3.0078493281054324

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph
value(min/max/mean) = -4.604097877510423 8.636466477209188 -2.470956381734402

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph
value(min/max/mean) = -4.604383714027222 8.765460701302153 -2.523270

In [7]:
# ЯЧЕЙКА (пример: 7). Расчёт предиктов Борзой (preds_borzoi_df) с учётом хромосомы.

preds_borzoi = []

for _, row in tqdm(labels_transcripts_df.iterrows(), 
                  total=labels_transcripts_df.shape[0], 
                  desc="Borzoi coverage"):
    
    t_id   = row["Transcript stable ID"]
    did    = row["description_id"]
    ideal  = row["preds"]  # логарифм идеальных меток (по условию уже лог)

    # Ищем [chrom, start, end] в labels_clear_df
    sub = labels_clear_df[labels_clear_df["Transcript stable ID"] == t_id]
    if sub.empty:
        # Нет записи -> NaN
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Возьмем первую строку или объединяем несколько (если нужно). 
    # Предположим, берём первую
    chrom  = sub.iloc[0]["chrom"]
    start_ = sub.iloc[0]["start"]
    end_   = sub.iloc[0]["end"]
    
    # По description_id ищем bedGraph-файл
    bedfname = experiment_mapping.get(did, None)
    if bedfname is None:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Достаем DataFrame (логарифмированный) из словаря
    df_cov = bedgraph_data_log.get(bedfname, None)
    if df_cov is None:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Фильтруем по той же хромосоме и интервалу [start..end].
    region = df_cov[
        (df_cov["chrom"] == chrom) &
        (df_cov["start"] >= start_) & 
        (df_cov["end"]   <= end_)
    ]
    
    if region.empty:
        # Если регион пуст, интерпретируем как coverage = 0 => log(0+pseudocount) => 
        # но у нас уже "bedgraph_data_log" логарифмирован. Значит лог(0+pseudocount) ~ какое-то маленькое число.
        # Выберем просто 0.0
        preds_borzoi.append([t_id, did, 0.0, ideal])
        continue
    
    # Усредняем лог-покрытия
    mean_logcov = region["value"].mean()
    
    preds_borzoi.append([t_id, did, mean_logcov, ideal])

# Собираем в DataFrame
preds_borzoi_df = pd.DataFrame(preds_borzoi, 
                               columns=["Transcript stable ID","description_id","pred_borzoi","pred_ideal"])


Borzoi coverage: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22095/22095 [3:20:05<00:00,  1.84it/s]


In [8]:
# Выводим часть результата
display(preds_borzoi_df.head(30))

print("pred_borzoi min/max/mean:",
      preds_borzoi_df["pred_borzoi"].min(),
      preds_borzoi_df["pred_borzoi"].max(),
      preds_borzoi_df["pred_borzoi"].mean())
print("pred_ideal min/max/mean:",
      preds_borzoi_df["pred_ideal"].min(),
      preds_borzoi_df["pred_ideal"].max(),
      preds_borzoi_df["pred_ideal"].mean())

,Transcript stable ID,description_id,pred_borzoi,pred_ideal
0,ENST00000001008,ENCFF123KIW,4.712871,4.037295
1,ENST00000002596,ENCFF123KIWr,1.099219,4.281860
2,ENST00000002829,ENCFF123KIW,3.475321,5.388129
3,ENST00000009180,ENCFF123KIW,6.688652,7.723541
4,ENST00000009589,ENCFF123KIWr,5.853962,8.866660
5,ENST00000011653,ENCFF123KIW,0.876107,5.424859
6,ENST00000011691,ENCFF123KIW,3.293093,-0.916062
7,ENST00000011898,ENCFF123KIW,1.479862,5.960806
8,ENST00000019103,ENCFF123KIWr,3.211000,4.228887
9,ENST00000027335,ENCFF123KIWr,0.262607,-2.936408


pred_borzoi min/max/mean: -4.578855155245346 8.14484626177008 0.19121216321998807
pred_ideal min/max/mean: -4.605170200000001 11.856355566803279 -0.4990853600718094


In [9]:
# Объединим preds_borzoi_df c preds_2seg/4seg/8seg + идеальные метки

# Начнём с preds_borzoi_df
df_merged = preds_borzoi_df.copy()

# preds_2seg_df -> добавим столбец preds_2seg
temp = preds_2seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_2"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

# preds_4seg_df -> preds_4seg
temp = preds_4seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_4"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

# preds_8seg_df -> preds_8seg
temp = preds_8seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_8"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

print("df_merged final shape:", df_merged.shape)
display(df_merged.head())

# Выведем статистику
for col in ["pred_borzoi","pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8"]:
    if col in df_merged.columns:
        print(f"{col} min={df_merged[col].min()} max={df_merged[col].max()} mean={df_merged[col].mean()}")
    else:
        print(f"Столбец {col} не найден.")

# Считаем корреляции (Pearson для наглядности)
corr_cols = ["pred_borzoi","pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8"]
df_corr = df_merged[corr_cols].corr(method="pearson")
print("\nКорреляции (Pearson):")
display(df_corr)


df_merged final shape: (22095, 7)


,Transcript stable ID,description_id,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8
0,ENST00000001008,ENCFF123KIW,4.712871,4.037295,0.570057,2.727709,3.736615
1,ENST00000002596,ENCFF123KIWr,1.099219,4.281860,-2.881656,0.732307,1.674160
2,ENST00000002829,ENCFF123KIW,3.475321,5.388129,1.192969,2.426938,0.736134
3,ENST00000009180,ENCFF123KIW,6.688652,7.723541,2.320119,4.060136,4.146430
4,ENST00000009589,ENCFF123KIWr,5.853962,8.866660,4.619201,4.862711,5.075033


pred_borzoi min=-4.578855155245346 max=8.14484626177008 mean=0.19121216321998807
pred_ideal min=-4.605170200000001 max=11.856355566803279 mean=-0.4990853600718094
pred_rmt_2 min=-3.8326391907692305 max=6.641538864634146 mean=-1.7986488347158898
pred_rmt_4 min=-4.2842083040022745 max=7.001022550393701 mean=-0.715264122164497
pred_rmt_8 min=-4.180466452112676 max=6.389540544094489 mean=-0.6615956298938233

Корреляции (Pearson):


,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8
pred_borzoi,1.000000,0.784860,0.657547,0.779853,0.782714
pred_ideal,0.784860,1.000000,0.666830,0.781221,0.778404
pred_rmt_2,0.657547,0.666830,1.000000,0.818327,0.773918
pred_rmt_4,0.779853,0.781221,0.818327,1.000000,0.960751
pred_rmt_8,0.782714,0.778404,0.773918,0.960751,1.000000


In [14]:
# ЯЧЕЙКА: Загрузка файла t_g.tsv и анализ характеристик

# Пути к файлу
t_g_file = os.path.join(RMT_INFER_DIR, "t_g.tsv")

# Чтение файла (TSV)
t_g_df = pd.read_csv(t_g_file, sep="\t")

# Выводим первые 5 строк для проверки структуры
print("Первые 5 строк t_g_df:")
display(t_g_df.head())

# Выводим информацию о столбцах и типах данных
print("\nИнформация о t_g_df:")
t_g_df.info()

# Выводим общую статистику по числовым столбцам
print("\nСтатистика числовых столбцов:")
display(t_g_df.describe())


Первые 5 строк t_g_df:


,Unnamed: 0,chrom,name,strand,strand.1,split,Transcript stable ID,start,end
0,29191,chr3,ENST00000002829,1,ENSG00000001617,valid,ENST00000002829,50140727,50189075
1,29192,chr12,ENST00000358495,-1,ENSG00000002016,valid,ENST00000358495,911735,964290
2,29193,chr4,ENST00000226299,1,ENSG00000002549,valid,ENST00000226299,17562601,17607970
3,29194,chr4,ENST00000002596,-1,ENSG00000002587,valid,ENST00000002596,11393149,11443490
4,29195,chr2,ENST00000264276,-1,ENSG00000003393,valid,ENST00000264276,201700266,201795529



Информация о t_g_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4419 entries, 0 to 4418
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            4419 non-null   int64 
 1   chrom                 4419 non-null   object
 2   name                  4419 non-null   object
 3   strand                4419 non-null   int64 
 4   strand.1              4419 non-null   object
 5   split                 4419 non-null   object
 6   Transcript stable ID  4419 non-null   object
 7   start                 4419 non-null   int64 
 8   end                   4419 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 310.8+ KB

Статистика числовых столбцов:


,Unnamed: 0,strand,start,end
count,4419.000000,4419.000000,4.419000e+03,4.419000e+03
mean,31400.000000,0.014709,8.914693e+07,8.920866e+07
std,1275.799749,1.000005,6.921697e+07,6.921950e+07
min,29191.000000,-1.000000,3.660100e+04,5.272900e+04
25%,30295.500000,-1.000000,2.567775e+07,2.573215e+07
50%,31400.000000,1.000000,8.228407e+07,8.236407e+07
75%,32504.500000,1.000000,1.298615e+08,1.299086e+08
max,33609.000000,1.000000,2.420874e+08,2.421406e+08


In [12]:
# ЯЧЕЙКА A: Загрузка gene-level TPM (ENCODE) для каждого эксперимента

encode_gene_tpm_files = {
    "ENCSR071DYD": "ENCFF258PUQ ENCSR071DYD.tsv",
    "ENCSR763OMY": "ENCFF993FDD ENCSR763OMY.tsv",
    "ENCSR357BYU": "ENCFF296APA ENCSR357BYU.tsv",
    "ENCSR892LBU": "ENCFF653OLW ENCSR892LBU.tsv",
    "ENCSR045GTF": "ENCFF769IPE ENCSR045GTF.tsv"
}

# Словарь experiment -> DataFrame (gene -> TPM)
encode_gene_tpm_data = {}

for experiment_id, fname in encode_gene_tpm_files.items():
    fpath = os.path.join(RMT_INFER_DIR, fname)
    if not os.path.exists(fpath):
        print(f"Файл TPM для эксперимента {experiment_id} не найден: {fname}")
        continue
    
    # Читаем TSV
    df_tpm = pd.read_csv(fpath, sep='\t')
    
    # Обычно в таких таблицах колонки: gene_id, TPM
    # У вас может быть "TPM" или "pme_TPM" и т.д. Нужно проверить.
    # Для примера возьмём столбец "TPM".
    
    # Выведем хотя бы head, чтобы убедиться в структуре
    print(f"\nЗагружена таблица TPM для {experiment_id} из {fname}, shape={df_tpm.shape}")
    display(df_tpm.head(3))
    
    # Сохраним df в словаре
    encode_gene_tpm_data[experiment_id] = df_tpm



Загружена таблица TPM для ENCSR071DYD из ENCFF258PUQ ENCSR071DYD.tsv, shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Загружена таблица TPM для ENCSR763OMY из ENCFF993FDD ENCSR763OMY.tsv, shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Загружена таблица TPM для ENCSR357BYU из ENCFF296APA ENCSR357BYU.tsv, shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Загружена таблица TPM для ENCSR892LBU из ENCFF653OLW ENCSR892LBU.tsv, shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



Загружена таблица TPM для ENCSR045GTF из ENCFF769IPE ENCSR045GTF.tsv, shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# ЯЧЕЙКА B: Группировка транскриптов по гену

gene_col = 'strand.1'  # предполагаем, что там ENSG...

# Преобразуем DataFrame preds_borzoi_df: добавим столбец gene_id
preds_borzoi_gene = pd.merge(
    preds_borzoi_df,        # transcript-level coverage
    t_g_df[['Transcript stable ID', gene_col]],  # сопоставление transcript-> gene
    on='Transcript stable ID',
    how='inner'
)

# Аналогично для RMT, если у вас df_merged содержит pred_rmt_2/4/8
preds_rmt_gene = pd.merge(
    df_merged,  # уже transcript-level (columns: pred_rmt_2/4/8)
    t_g_df[['Transcript stable ID', gene_col]],
    on='Transcript stable ID',
    how='inner'
)

print("preds_borzoi_gene shape:", preds_borzoi_gene.shape)
display(preds_borzoi_gene.head(3))

print("\npreds_rmt_gene shape:", preds_rmt_gene.shape)
display(preds_rmt_gene.head(3))

# Теперь группируем по gene_id + experiment (который можно извлечь из description_id?)
# description_id -> "ENCFF123KIW" -> experiment => "ENCSR892LBU" (через mapping)
def description_id_to_experiment(did):
    """Например, 'ENCFF123KIW' => 'ENCSR892LBU', 
       'ENCFF123KIWr' => 'ENCSR892LBU' 
       используя ваш mapping (либо обратный из experiment_mapping).
    """
    # нужно сделать reverse lookup experiment_mapping
    # но там '.tsv'...  упростим. 
    # Или вы заранее храните experiment:description_id в словаре. 
    # Для иллюстрации:
    if 'KIW' in did:
        return 'ENCSR892LBU'
    elif 'MDF' in did:
        return 'ENCSR357BYU'
    # и т.д...
    # Реализуйте по аналогии 
    # Возврат None если не нашлось
    return None

preds_borzoi_gene['experiment'] = preds_borzoi_gene['description_id'].apply(description_id_to_experiment)
preds_rmt_gene['experiment']    = preds_rmt_gene['description_id'].apply(description_id_to_experiment)

# Группировка transcript->gene: суммируем coverage (или усредняем).
# Например, усредним:
borzoi_gene_coverage = preds_borzoi_gene.groupby(
    ['experiment', gene_col], dropna=False
)['pred_borzoi'].mean().reset_index(name='gene_cov_borzoi')

# Для RMT 2/4/8seg
rmt_gene_coverage = preds_rmt_gene.groupby(
    ['experiment', gene_col], dropna=False
)[['pred_rmt_2','pred_rmt_4','pred_rmt_8']].mean().reset_index()

print("\nborzoi_gene_coverage:")
display(borzoi_gene_coverage.head(5))

print("\nrmt_gene_coverage:")
display(rmt_gene_coverage.head(5))


preds_borzoi_gene shape: (22095, 5)


,Transcript stable ID,description_id,pred_borzoi,pred_ideal,strand.1
0,ENST00000001008,ENCFF123KIW,4.712871,4.037295,ENSG00000004478
1,ENST00000002596,ENCFF123KIWr,1.099219,4.281860,ENSG00000002587
2,ENST00000002829,ENCFF123KIW,3.475321,5.388129,ENSG00000001617



preds_rmt_gene shape: (22095, 8)


,Transcript stable ID,description_id,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8,strand.1
0,ENST00000001008,ENCFF123KIW,4.712871,4.037295,0.570057,2.727709,3.736615,ENSG00000004478
1,ENST00000002596,ENCFF123KIWr,1.099219,4.281860,-2.881656,0.732307,1.674160,ENSG00000002587
2,ENST00000002829,ENCFF123KIW,3.475321,5.388129,1.192969,2.426938,0.736134,ENSG00000001617



borzoi_gene_coverage:


,experiment,strand.1,gene_cov_borzoi
0,ENCSR357BYU,ENSG00000001617,1.269415
1,ENCSR357BYU,ENSG00000002016,2.441396
2,ENCSR357BYU,ENSG00000002549,2.269052
3,ENCSR357BYU,ENSG00000002587,-0.903916
4,ENCSR357BYU,ENSG00000003393,4.305475



rmt_gene_coverage:


,experiment,strand.1,pred_rmt_2,pred_rmt_4,pred_rmt_8
0,ENCSR357BYU,ENSG00000001617,-0.592517,0.330861,-0.410446
1,ENCSR357BYU,ENSG00000002016,-0.704584,2.531929,2.007010
2,ENCSR357BYU,ENSG00000002549,2.039521,4.114191,4.202942
3,ENCSR357BYU,ENSG00000002587,-3.221785,-1.531423,-1.911324
4,ENCSR357BYU,ENSG00000003393,-0.349772,3.345377,3.081822
